# k-Nearest Neighbors

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
Boston = pd.read_table('Boston.txt', sep=',', dtype='float32')
Boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1.0,0.00632,18.0,2.31,0.0,0.538,6.575,65.199997,4.0900,1.0,296.0,15.300000,396.899994,4.98,24.000000
2.0,0.02731,0.0,7.07,0.0,0.469,6.421,78.900002,4.9671,2.0,242.0,17.799999,396.899994,9.14,21.600000
3.0,0.02729,0.0,7.07,0.0,0.469,7.185,61.099998,4.9671,2.0,242.0,17.799999,392.829987,4.03,34.700001
4.0,0.03237,0.0,2.18,0.0,0.458,6.998,45.799999,6.0622,3.0,222.0,18.700001,394.630005,2.94,33.400002
5.0,0.06905,0.0,2.18,0.0,0.458,7.147,54.200001,6.0622,3.0,222.0,18.700001,396.899994,5.33,36.200001


In [3]:
Boston.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 506 entries, 1.0 to 506.0
Data columns (total 14 columns):
crim       506 non-null float32
zn         506 non-null float32
indus      506 non-null float32
chas       506 non-null float32
nox        506 non-null float32
rm         506 non-null float32
age        506 non-null float32
dis        506 non-null float32
rad        506 non-null float32
tax        506 non-null float32
ptratio    506 non-null float32
black      506 non-null float32
lstat      506 non-null float32
medv       506 non-null float32
dtypes: float32(14)
memory usage: 31.6 KB


In [4]:
Boston.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613523,11.363636,11.136797,0.069170,0.554696,6.284636,68.574921,3.795043,9.549407,408.237152,18.455584,356.674561,12.653064,22.532806
std,8.601545,23.322390,6.860355,0.253993,0.115878,0.702617,28.148869,2.105711,8.707269,168.537170,2.164946,91.294838,7.141063,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377487,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440002,11.360000,21.200001
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.074999,5.188425,24.000000,666.000000,20.200001,396.225006,16.954999,25.000000
max,88.976196,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.899994,37.970001,50.000000


In [5]:
X_train = Boston.drop('medv', axis=1).values[:404,:]
Y_train = Boston.loc[:,['medv']].values[:404,:]

X_test = Boston.drop('medv', axis=1).values[404:,:]
Y_test = Boston.loc[:,['medv']].values[404:,:]

In [6]:
class Layer():
    
    def __init__(self, input_dim, n_clusters, k):
        self.input_dim = input_dim
        self.n_clusters = n_clusters
        self.k = k
        self.build()
        
    def add_weight(self, shape):
        weight_init = tf.random.normal(shape=shape, mean=0.0, stddev=0.05, dtype="float32")
        return tf.Variable(initial_value=weight_init, trainable=False)
        
    def build(self):
        self.x = self.add_weight(shape=(self.n_clusters, self.input_dim))
        self.y = self.add_weight(shape=(self.n_clusters, 1))
        self.weights = [self.x, self.y]
        
    def __call__(self, inputs): 
        n = self.n_clusters
        i = tf.repeat(inputs, repeats=n, axis=0)
        m = inputs.shape[0]
        u = tf.tile(self.x, [m,1])
        v = tf.reduce_sum(tf.square(u-i), axis=1)
        w = tf.reshape(v, shape=(m,n))
        _, indices = tf.math.top_k(-w, k=self.k)
        return tf.reduce_mean(tf.gather(self.y,indices), axis=1)

In [7]:
class LossFunction():
    
    def __init__(self, model):
        self.model = model
                    
    def __call__(self, y_true, y_pred):
        return tf.math.reduce_mean(tf.square(tf.sign(y_true-y_pred))) 

In [8]:
class MetricFunction():
    
    def __init__(self, model):
        self.model = model
                    
    def __call__(self, y_true, y_pred):
        return tf.math.reduce_mean(tf.math.abs(y_true-y_pred))

In [9]:
class Optimizer():

    def __init__(self, model):
        self.model = model
            
    def train_step(self, X, Y):
        self.model.weights[0].assign(X)
        self.model.weights[1].assign(Y)
        H = self.model(X)
        loss = self.model.loss(Y, H)
        metric = self.model.metric(Y, H)
        logs = {'loss': loss,
                'metric': metric}
        return logs

In [10]:
class Callback():

    def __init__(self, model, verbose):
        self.model = model
        self.verbose = verbose
    
    def on_epoch_begin(self, logs=None):
        self.start_time = tf.timestamp()
        
    def on_epoch_end(self, logs=None):
        if self.verbose:
            now = tf.timestamp()
            time = now - self.start_time
            tf.print('Loss: {} - Metric: {}'.format(logs['loss'], logs['metric']))
            tf.print('----- {}s -----'.format(tf.round(1000*time)/1000))

In [11]:
class KNN():
    
    def __init__(self, input_dim, n_clusters, k):
        self.input_dim = input_dim
        self.n_clusters = n_clusters
        self.k = k
        self.build()
    
    def build(self):
        self.h = Layer(self.input_dim, self.n_clusters, self.k)
        self.layers = [self.h]
        self.weights = []
        for layer in self.layers:
            for weight in layer.weights:
                self.weights.append(weight)
        
    def __call__(self, inputs):
        y = self.h(inputs)
        return y
        
    def train_setup(self, verbose):
        self.verbose = verbose
        self.loss = LossFunction(model=self)
        self.metric = MetricFunction(model=self)
        self.optimizer = Optimizer(model=self) 
        self.callbacks = [Callback(model=self, verbose=self.verbose)]
        
    def fit(self, X, Y, verbose=True):
        self.train_setup(verbose)
        if verbose:
            print('Train on {} samples'.format(X.shape[0]))
        self.callbacks[0].on_epoch_begin()
        logs = self.optimizer.train_step(tf.constant(X, dtype="float32"), tf.constant(Y, dtype="float32"))
        self.callbacks[0].on_epoch_end(logs=logs)
            
    def predict(self, inputs):
        return self(tf.constant(inputs, dtype="float32")).numpy()
    
    def evaluate(self, X, Y):
        loss = self.loss(tf.constant(Y, dtype="float32"), self(tf.constant(X, dtype="float32")))
        loss_numpy = loss.numpy()
        metric = self.metric(tf.constant(Y, dtype="float32"), self(tf.constant(X, dtype="float32")))
        metric_numpy = metric.numpy()
        tf.print('Loss: {} - Metric: {}'.format(loss_numpy, metric_numpy))
        return [loss_numpy, metric_numpy]

In [12]:
knn_1 = KNN(input_dim=X_train.shape[1], n_clusters=X_train.shape[0], k=1)

knn_1.fit(X_train, Y_train)

Train on 404 samples
Loss: 0.0 - Metric: 0.0
----- 0.034s -----


In [13]:
knn_1.evaluate(X_test, Y_test)

Loss: 0.9901960492134094 - Metric: 6.66568660736084


[0.99019605, 6.6656866]

In [14]:
knn_2 = KNN(input_dim=X_train.shape[1], n_clusters=X_train.shape[0], k=3)

knn_2.fit(X_train, Y_train)

Train on 404 samples
Loss: 0.9950494766235352 - Metric: 3.1383659839630127
----- 0.011s -----


In [15]:
knn_2.evaluate(X_test, Y_test)

Loss: 1.0 - Metric: 4.59019660949707


[1.0, 4.5901966]